In [ ]:
# Google API 키
API_KEY = 

In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm

df_yn = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/yeonnam_list.csv")

# Google Maps API 요청 함수
def get_place_id(query):
    """검색어로 place_id와 가게명 얻기"""
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": query,
        "inputtype": "textquery",
        "fields": "place_id,name",
        "key": API_KEY
    }
    res = requests.get(url, params=params).json()
    candidates = res.get("candidates")
    if candidates:
        return candidates[0]["place_id"], candidates[0]["name"]
    return None, None

def get_info(place_id):
    """place_id로 상세정보 얻기"""
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "key": API_KEY,
        "language": "ko"
    }
    res = requests.get(url, params=params).json()
    if res["status"] == "OK":
        result = res["result"]
        return {
            "검색결과 가게명": result.get("name", ""),
            "평점": result.get("rating", ""),
            "리뷰 수": result.get("user_ratings_total", "")
        }
    return {}

# 정보 수집
all_infos = []

for i, row in tqdm(df_yn.iterrows(), total=len(df_yn)):
    search_query = row["검색어2"]
    store_name = row["사업장명"]
    
    try:
        place_id, place_name = get_place_id(search_query)
        if place_id:
            info = get_info(place_id)
            info["검색어"] = search_query
            info["사업장명"] = store_name
            all_infos.append(info)
        else:
            print(f"[미발견] {search_query}")
        time.sleep(0.3)  # 너무 빠른 요청 방지
    except Exception as e:
        print(f"[오류] {search_query}: {e}")
        continue

# 결과 저장
df_infos = pd.DataFrame(all_infos)
df_infos.to_csv("gmaps_info_api.csv", index=False, encoding="utf-8-sig")
print("수집 완료 및 저장 완료")


 35%|███▍      | 206/591 [03:45<06:33,  1.02s/it]

[미발견] 03980 545


 47%|████▋     | 279/591 [05:02<05:38,  1.08s/it]

[미발견] 03989 삼육식당


100%|██████████| 591/591 [10:40<00:00,  1.08s/it]

수집 완료 및 저장 완료


In [2]:
import pandas as pd

In [17]:
df_info = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/gmaps_info_api.csv")

df_info.columns

Index(['검색결과 가게명', '평점', '리뷰 수', '검색어', '사업장명'], dtype='object')

In [18]:
df_info.drop(columns=['검색어'], inplace=True)

new_order = ['사업장명', '검색결과 가게명', '평점', '리뷰 수']
df_info = df_info[new_order]

df_info.head()

,사업장명,검색결과 가게명,평점,리뷰 수
0,연남술비,suulbi,5.0,24.0
1,유메오뎅,Yume Fish Cake Yeonnam,4.1,10.0
2,헤비메탈,Heavy Metal,5.0,15.0
3,문일리(moonily),MOONILY PIZZA HOUSE,4.4,21.0
4,스필아웃,Spill Out (Pizza & Bar),4.9,136.0


In [23]:
df_info.to_csv("gmaps_infos_api.csv", index=False, sep="\t", encoding="utf-8-sig")